In [10]:
import cv2
import numpy as np
import random
import time
from deepface import DeepFace
from mtcnn.mtcnn import MTCNN

# Models load and setup
haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
mtcnn_det = MTCNN()

# Load emojis for each emotion
emojis = {
    "happy": cv2.imread("images/happy.png", cv2.IMREAD_UNCHANGED),
    "sad": cv2.imread("images/sad.png", cv2.IMREAD_UNCHANGED),
    "fear": cv2.imread("images/fear.png", cv2.IMREAD_UNCHANGED),
    "neutral": cv2.imread("images/neutral.png", cv2.IMREAD_UNCHANGED),
    "disgust": cv2.imread("images/disgust.png", cv2.IMREAD_UNCHANGED),
    "angry": cv2.imread("images/angry.png", cv2.IMREAD_UNCHANGED),
    "surprise": cv2.imread("images/surprised.png", cv2.IMREAD_UNCHANGED)
}

# Define emotions and instructions
emotions = list(emojis.keys())
score = 0
countdown_time = 5

# Webcam connection
video_capture = cv2.VideoCapture(0)

# Game variables
current_emotion = random.choice(emotions)
instruction_text = f"Simon says: Be {current_emotion}"
start_time = time.time()
trap_instructions = [True, False]  # True: "Simon says", False: trap (without "Simon says")

# Main loop
while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # Calculate elapsed time for countdown
    elapsed_time = time.time() - start_time
    remaining_time = countdown_time - int(elapsed_time)

    # Reset the game after each countdown
    if remaining_time <= 0:
        remaining_time = countdown_time
        start_time = time.time()
        
        # Decide if the next instruction is a trap
        is_trap = random.choice(trap_instructions)
        current_emotion = random.choice(emotions)
        if is_trap:
            instruction_text = f"Be {current_emotion}"
        else:
            instruction_text = f"Simon says: Be {current_emotion}"

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Analyze the face for emotion and overlay emoji
    detected_emotion = None
    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        try:
            # Analyze emotion with DeepFace
            analysis = DeepFace.analyze(img_path=face_img, enforce_detection=False, actions=['emotion'])
            detected_emotion = analysis['dominant_emotion']

            # Overlay the corresponding emoji on the face
            if detected_emotion in emojis:
                emoji = emojis[detected_emotion]
                emoji_resized = cv2.resize(emoji, (w, h))

                # Overlay emoji with transparency
                for i in range(h):
                    for j in range(w):
                        if emoji_resized[i, j][3] != 0:  # Check the alpha channel for transparency
                            frame[y + i, x + j] = emoji_resized[i, j][:3]
            
            # Check if detected emotion matches the current instruction
            if detected_emotion == current_emotion:
                if "Simon says" in instruction_text:
                    score += 1  # Correct response
                else:
                    score -= 1  # Incorrect response due to trap
                remaining_time = countdown_time  # Reset countdown
                start_time = time.time()
                # Choose a new random emotion and instruction
                current_emotion = random.choice(emotions)
                is_trap = random.choice(trap_instructions)
                instruction_text = f"Be {current_emotion}" if is_trap else f"Simon says: Be {current_emotion}"

        except Exception as e:
            print(f"Emotion detection failed for face at {(x, y, w, h)}: {e}")

    # Display the current "Simon says" instruction
    cv2.rectangle(frame, (0, frame.shape[0] - 50), (frame.shape[1], frame.shape[0]), (255, 255, 255), -1)
    cv2.putText(frame, instruction_text, (10, frame.shape[0] - 15), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # Draw countdown circle in top right corner
    center = (frame.shape[1] - 60, 60)
    radius = 30
    cv2.circle(frame, center, radius, (169, 169, 169), -1)
    cv2.putText(frame, str(remaining_time), (frame.shape[1] - 75, 75), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Display score in top left corner
    cv2.putText(frame, f"Score: {score}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Show the result
    cv2.imshow('Simon Says Emotion Game', frame)

    # Exit on 'Esc' key
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release the camera and close all windows
video_capture.release()
cv2.destroyAllWindows()


Emotion detection failed for face at (np.int32(221), np.int32(192), np.int32(146), np.int32(146)): list indices must be integers or slices, not str
Emotion detection failed for face at (np.int32(232), np.int32(203), np.int32(123), np.int32(123)): list indices must be integers or slices, not str
Emotion detection failed for face at (np.int32(227), np.int32(204), np.int32(126), np.int32(126)): list indices must be integers or slices, not str
Emotion detection failed for face at (np.int32(228), np.int32(205), np.int32(125), np.int32(125)): list indices must be integers or slices, not str
Emotion detection failed for face at (np.int32(228), np.int32(204), np.int32(126), np.int32(126)): list indices must be integers or slices, not str
Emotion detection failed for face at (np.int32(231), np.int32(204), np.int32(128), np.int32(128)): list indices must be integers or slices, not str
Emotion detection failed for face at (np.int32(231), np.int32(203), np.int32(128), np.int32(128)): list indices 

Make a Simon Said game to change the emotions , give a 5 seconds timer. Only if Simon says it you change the emotion, and get a point, otherwise you lose a point